In [ ]:
import overpy
import time
import os
import numpy as np
import xml.etree.ElementTree as ET
from urllib.error import URLError

error_bbox_file = open(r"D:\近期需要做的\shp数据\OSM_new\error_bboxes1.txt", "w")  # 在代码开始时打开一个文件

ENDPOINTS = [
    "https://overpass-api.de/api/interpreter",
    "https://maps.mail.ru/osm/tools/overpass/api/interpreter",
    "https://overpass.openstreetmap.ru/api/interpreter",
    "https://overpass.osm.ch/api/interpreter",
    "https://overpass.kumi.systems/api/interpreter",
    "https://z.overpass-api.de/api/interpreter",
    "https://lz4.overpass-api.de/api/interpreter",
]
current_endpoint_index = 0

api = overpy.Overpass(url=ENDPOINTS[current_endpoint_index])

MAX_RETRIES = 5

def result_to_osm_xml(result, data_type):
    osm = ET.Element("osm", version="0.6", generator="overpy")
    ALL_TAGS = [
        "osm_type", "water","type","natural","name","name:zh","name:en", "leisure", "landsure", "building:material", "building", 
        "source:zoomlevel", "source:tracer", "source:position", "wetland", "water_type",
        "source:name","place","gns:dsg","tidal","alt_name","old_name","int_name","ele","name:zh-Hant",
        "name:zh-Hans","alt_name:en","salt","intermittent","boat","addr:housename","wikipedia","wikidata",
        "name:zh_pinyin","code","alias","source_ref"
    ]
    if data_type == "node":
        for node in result.nodes:
            node_elem = ET.SubElement(osm, "node", id=str(node.id), lat=str(node.lat), lon=str(node.lon))
            for tag in ALL_TAGS:
                value = node.tags.get(tag)
                if value:
                    ET.SubElement(node_elem, "tag", k=tag, v=str(value))

    # 处理多边形（包括 way 和 relation）
    elif data_type == "polygon":
        # 首先处理所有的 way
        for way in result.ways:
            way_elem = ET.SubElement(osm, "way", id=str(way.id))
            for node in way.nodes:
                ET.SubElement(way_elem, "nd", ref=str(node.id))
                # 也包括节点的详细信息
                node_elem = ET.SubElement(osm, "node", id=str(node.id), lat=str(node.lat), lon=str(node.lon))
                for tag in ALL_TAGS:
                    value = node.tags.get(tag)
                    if value:
                        ET.SubElement(node_elem, "tag", k=tag, v=str(value))
            for tag in ALL_TAGS:
                value = way.tags.get(tag)
                if value:
                    ET.SubElement(way_elem, "tag", k=tag, v=str(value))

        # 处理 relation
        for relation in result.relations:
            relation_elem = ET.SubElement(osm, "relation", id=str(relation.id))
            for member in relation.members:
                m_type = "node" if isinstance(member, overpy.RelationNode) else "way"
                ET.SubElement(relation_elem, "member", type=m_type, ref=str(member.ref), role=member.role)
            for tag in ALL_TAGS:
                value = relation.tags.get(tag)
                if value:
                    ET.SubElement(relation_elem, "tag", k=tag, v=str(value))

    return ET.tostring(osm, encoding="utf-8").decode("utf-8")

def switch_endpoint():
    global current_endpoint_index, api
    current_endpoint_index = (current_endpoint_index + 1) % len(ENDPOINTS)
    api = overpy.Overpass(url=ENDPOINTS[current_endpoint_index])

def query_bbox(min_lat, min_lon, max_lat, max_lon):
    query_string = f"""
    [out:xml][timeout:10000];
    (
      // 查询水库（reservoir）的点和多边形
      node["natural"="water"]["water"="reservoir"]({min_lat},{min_lon},{max_lat},{max_lon});
      way["natural"="water"]["water"="reservoir"]({min_lat},{min_lon},{max_lat},{max_lon});
      relation["natural"="water"]["water"="reservoir"]({min_lat},{min_lon},{max_lat},{max_lon});
      // 查询大坝（dam）的点和多边形
      node["waterway"="dam"]({min_lat},{min_lon},{max_lat},{max_lon});
      way["waterway"="dam"]({min_lat},{min_lon},{max_lat},{max_lon});
      relation["waterway"="dam"]({min_lat},{min_lon},{max_lat},{max_lon});
      // 查询使用 landuse=reservoir 标记的水库
      node["landuse"="reservoir"]({min_lat},{min_lon},{max_lat},{max_lon});
      way["landuse"="reservoir"]({min_lat},{min_lon},{max_lat},{max_lon});
      relation["landuse"="reservoir"]({min_lat},{min_lon},{max_lat},{max_lon});
    );
    out body;
    >;
    out skel qt;
    """
    return api.query(query_string)

def query_bbox_with_retry(min_lat, min_lon, max_lat, max_lon, max_retries=3):
    retries = 0
    while retries < max_retries:
        try:
            return query_bbox(min_lat, min_lon, max_lat, max_lon)
        except URLError as e:
            print(f"Error: {e}, retrying in 5 seconds...")
            time.sleep(5)
            retries += 1
    raise Exception("Max retries reached, aborting.")

start_lat, start_lon = -90, -180
for lat in np.arange(start_lat, 90, 2):
    for lon in np.arange(start_lon, 180, 2):
        print(f"Processing bbox: {lat},{lon},{lat+2},{lon+2}...")
        success = False
        attempt = 0
        while not success and attempt < MAX_RETRIES:
            try:
                result = query_bbox_with_retry(lat, lon, lat+2, lon+2)

                if not result.nodes and not result.ways and not result.relations:
                    print(f"No reservoir or dam data for bbox: {lat},{lon},{lat+2},{lon+2}.")
                    break

                # 检查是否有点数据
                if result.nodes:
                    print("Processing and saving node data...")
                    xml_data = result_to_osm_xml(result, "node")
                    filename = f"nodes_{lat}_{lon}_{lat+2}_{lon+2}.osm"
                    filepath = os.path.join(r"D:\近期需要做的\shp数据\OSM_new\new_OSM", filename)
                    with open(filepath, 'w', encoding='utf-8') as f:
                        f.write(xml_data)

                # 检查是否有多边形数据
                if result.ways or result.relations:
                    print("Processing and saving polygon data...")
                    xml_data = result_to_osm_xml(result, "polygon")
                    filename = f"polygons_{lat}_{lon}_{lat+2}_{lon+2}.osm"
                    filepath = os.path.join(r"D:\近期需要做的\shp数据\OSM_new\new_OSM", filename)
                    with open(filepath, 'w', encoding='utf-8') as f:
                        f.write(xml_data)

                success = True
                print(f"Successfully processed bbox: {lat},{lon},{lat+2},{lon+2}")

            except overpy.exception.OverpassTooManyRequests as e:
                print(f"Server {ENDPOINTS[current_endpoint_index]} busy, switching endpoint...")
                switch_endpoint()
                print(f"Switched to endpoint: {ENDPOINTS[current_endpoint_index]}")
                attempt += 1

            except Exception as e:
                print(f"Failed to fetch data for bbox: {lat},{lon},{lat+2},{lon+2}. Error: {e}")
                attempt += 1
                if attempt >= MAX_RETRIES:
                    print(f"Max retries reached for bbox: {lat},{lon},{lat+2},{lon+2}, writing to error file...")
                    error_bbox_file.write(f"{lat},{lon},{lat+2},{lon+2}\n")
                time.sleep(5)
    start_lon = -180  # 重置经度，以便下一纬度行从0开始

print("Download complete!")
error_bbox_file.close()  # 不要忘记关闭文件